In [5]:
import cv2

import numpy as np
import pandas as pd
from scipy.spatial import distance
from tqdm import tqdm

In [6]:
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32

PATH_PREFIX_1 = '../data/letters/'
PATH_PREFIX_2 = '../data/letters2/'
PATH_PREFIX_3 = '../data/letters3/'

# абвгдеёжзийклмнопрстуфхцчшщъыьэюя
russian_alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [7]:
class BAM:
    def __init__(self, train_df):
        self.pairs = train_df

        self.len_x = IMAGE_HEIGHT
        self.len_y = IMAGE_WIDTH

        assert self.len_x > 0
        assert self.len_y > 0

        self.kernel = np.zeros(shape=(self.len_y, self.len_x), dtype='uint8')
        self.make_association_matrix()

    def make_association_matrix(self):
        for vector in tqdm(self.pairs, desc='Building kernel'):
            x = np.array(vector[0])
            y = np.array(vector[1])

            self.kernel = np.add(self.kernel, np.dot(x.T[:, None], y[None]))
        print('MAX: ', np.max(self.kernel))

    def solve(self, input_vector):
        input_vector = input_vector.dot(self.kernel)
        return self.compute_threshold(input_vector)

    def get_kernel(self):
        return self.kernel

    @staticmethod
    def compute_threshold(vector) -> np.array:
        return np.array([0 if x < 0 else 1 for x in vector])

    @staticmethod
    def convert_to_bipolar(_image) -> np.array:
        return np.array([-1 if x == 0 else 1 for x in _image])  


In [8]:
print(np.__config__.show())

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    library_dirs = ['C:\\projects\\numpy-wheels\\numpy\\build\\openblas_info']
    libraries = ['openblas_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    library_dirs = ['C:\\projects\\numpy-wheels\\numpy\\build\\openblas_info']
    libraries = ['openblas_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    library_dirs = ['C:\\projects\\numpy-wheels\\numpy\\build\\openblas_lapack_info']
    libraries = ['openblas_lapack_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    library_dirs = ['C:\\projects\\numpy-wheels\\numpy\\build\\openblas_lapack_info']
    libraries = ['openblas_lapack_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
None


In [9]:
def compute_distances(target, associations):
    distances = distance.cdist(
        [target],
        np.array(
            [[1 if x == 255 else 0 for x in y] for y in associations], dtype='uint8'
        ),
        "cosine"
    )[0]
    min_index = np.argmin(distances)
    min_distance = distances[min_index]
    max_similarity = 1 - min_distance

    return max_similarity

In [10]:
def open_image(image_path) -> np.array:
    # load image with alpha channel use IMREAD_UNCHANGED to ensure loading of alpha channel
    _image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    # make mask of where the transparent bits are
    trans_mask = _image[:, :, 3] == 0

    # replace areas of transparency with white and not transparent
    _image[trans_mask] = [255, 255, 255, 255]

    # new image without alpha channel...
    new_img = cv2.cvtColor(_image, cv2.COLOR_BGRA2BGR)
    new_img = cv2.resize(new_img, (IMAGE_HEIGHT, IMAGE_WIDTH))

    return np.array(new_img, dtype='uint8')

In [11]:
def make_pair(input_matrix):
    return np.array([
        BAM.convert_to_bipolar(input_matrix[0]),
        BAM.convert_to_bipolar(input_matrix[1])
    ])


In [12]:

dataset = pd.read_csv('../data/letters.csv')
dataset.head()


,letter,label,file,background
0,а,1,01_01.png,0
1,а,1,01_02.png,0
2,а,1,01_03.png,0
3,а,1,01_04.png,0
4,а,1,01_05.png,0


In [ ]:
# bam = BAM(dataset['file'])


In [24]:
images_path = dataset['file']

a_i = np.array([])
b_i = np.array([])

for index, path in enumerate(images_path):
    image = open_image(PATH_PREFIX_1 + path)
    
    if path[3:5:] == '01':
        perfect = images_path[index]
        b_i = np.concatenate((b_i, image.ravel()))   
    else:
        a_i = np.concatenate((
            a_i,
            make_pair([image.ravel(), b_i[int(path[:2:])]])
        ))

TypeError: 'numpy.float64' object is not iterable

In [ ]:
bam = BAM(a_i)

print(bam.solve(PATH_PREFIX_1 + '24_34.png'))
print(russian_alphabet[25])
